In [1]:
# This example is from https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/
# It uses an RNN to predict the next step in binary addition

# Import dependencies and seed the random number generator
import copy, numpy as np
np.random.seed(0)


In [2]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

In [3]:
# training dataset generation
# empty list that maps from an integer to its binary representation
int2binary = {}
# set max length of binary numbers
binary_dim = 8

In [12]:
# computes the largest number that is possible to represent using binary_dim
largest_number = pow(1, binary_dim)
#print(largest_number)
# a look up table that maps from an int to its binary. We copy this into the 
# int2binary
binary = np.unpackbits(
    np.array([range(largest_number)], dtype=np.uint8).T, axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]
print(int2binary)
#print(binary)

{0: array([0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)}


In [13]:
# set valuyes for input variables
# learning rate
alpha = 0.1
# number of inputs (adding 2 numbers together)
input_dim = 2
# number of hidden layers
hidden_dim = 16
# number of outputs (predictin the sum)
output_dim = 1

In [23]:
# matrix of weights that connects the input and hidden layer (2,16)
synapse_0 = 2*np.random.random((input_dim, hidden_dim)) - 1
#print(synapse_0)
#synapse_0.shape

# matrix of weights that connects the hidden layers to the output layers (16,1)
synapse_1 = 2*np.random.random((hidden_dim, output_dim)) - 1
#synapse_1.shape

# matrix of weights that connects the hidden layer in the previous time-step
# to the current timestep (16,16)
synapse_h = 2*np.random.random((hidden_dim, hidden_dim)) - 1
#synapse_h.shape


(16, 16)

In [25]:
# these store the weight updates that we make for each of the weight matricies
synapse_0_update = np.zeros_like(synapse_0)
#print(synapse_0_update)
#synapse_0_update.shape
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

In [30]:
 # Training logic, iterate over 100000 examples
for j in range(100000):
    # generate an addition problem a+b=c
    # initialize an integer with a random number that cannot be more 
    # than half of the largest number
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding
    b_int = np.random.randint(largest_number/2) #int version
    b = int2binary[b_int] # binary encoding
    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    # where we'll store our best guess, the NN's predictions(binary encoded)
    d = np.zeros_like(c)
    # resetting the error measure(which we use as a means to track convergence)
    overallError = 0
    # keep track of the layer 2 derivatives and layer 1 values at each step
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    # This loop iterates through the binary representation
    for position in range(binary_dim):
        # generate input and output, X is a list of 2 numbers, one from a and b
        # we index it so it goes from right to left, when position == 0, this is 
        # the farthest bit to the right, when position==1 this shifts to the left
        # by one bit
        X = np.array([[a[binary_dim - position -1], 
                       b[binary_dim - position - 1]]])
        # y is the value of the correct answer (either 1 or 0)
        y = np.array([[c[binary_dim - position - 1]]]).T
        # hidden layer (sum the input and prev_hidden and pass through sigmoid func)
        layer_1 = sigmoid(np.dot(X, synapse_0) + np.dot(layer_1_values[-1], synapse_h))
        # output layer (new binary representation)
        # propagates the hidden layer to the output to make a prediction
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))
        # Compute how much the prediction  missed
        layer_2_error = y - layer_2
        # store the derivative in a list, holding the derivative at each time step
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        # Calculate the sum of the abs errors 
        overallError +=np.abs(layer_2_error[0])
        # Round the output to a binary and stores in d
        d[binary_dim - position -1] = np.round(layer_2[0][0])
        # store hidden layer so we can use it in the next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
        
    future_layer_1_delta = np.zeros(hidden_dim)
    # Now to back propagate starting with the last to the first
    for position in range(binary_dim):
        # index the input data
        X = np.array([[a[position], b[position]]])
        # select current hidden layer from the list
        layer_1 = layer_1_values[-position-1]
        # select the previous hidden layer from the list
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
        # select the current output error from the list
        layer_2_delta = layer_2_deltas[-position-1]
        # computes the current hidden layer error given the error at the hidden
        # layer from the future and the error at the current output
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + 
                        layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)
        # update all the weights so we can try again
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    # Time to update weights and empty the update variables
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha
    
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # print out progress
    if(j % 1000 == 0):
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c)) 
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("-------------")
        
        
        
        

Error:[ 5.21616617]
Pred:[1 1 1 1 1 1 1 1]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 255
-------------
Error:[ 0.08091101]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.05543493]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.04445174]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.03801134]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.03366822]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.03049211]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.02804258]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.02608096]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.02446535]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 0.02310547]
Pred:[0 0 0 0 0 0 0 0]
True:[0 0 0 0 0 0 0 0]
0 + 0 = 0
-------------
Error:[ 